In [2]:
import sqlite3

In [4]:
def main():
    print("SQLite example")
    db = None #Satisfy the warnings monster
    cur = None

    try:
        #Using SQLite's transient in-memory database
        db = sqlite3.connect(":memory:")
        cursor = db.cursor()
        print("Connected.")
    except sqlite3.Error as e:
        print(f'could not open database: {e}')
        exit(1)

    try:
        #Create a table
        sql_statement = '''
            CREATE TABLE IF NOT EXISTS hello (
                id INTEGER PRIMARY KEY,
                a TEXT,
                b TEXT,
                c TEXT
            )
        '''

        cursor.execute(sql_statement)
        print("Table created.")

    except sqlite3.Error as e:
        print(f'could not create table: {e}')
        exit(1)

    try:
        #Insert rows into the table using executemay
        print("Inserting rows.")
        row_data = (
            ('one', 'two', 'three'),
            ('two', 'three', 'four'),
            ('three', 'four', 'five'),
            ('four', 'five', 'six'),
            ('five', 'six', 'seven'),
            ('six', 'seven', 'eight'),
            ('seven', 'eight', 'nine'),
            ('eight', 'nine', 'ten'),
            ('nine', 'ten', 'eleven')
        )
        cursor.executemany("INSERT INTO hello (a, b, c) VALUES (?, ?, ?)", row_data)
        count = cursor.rowcount
        cursor.executemany("INSERT INTO hello (a, b, c) VALUES (?, ?, ?)", row_data)
        count += cursor.rowcount
        cursor.executemany("INSERT INTO hello (a, b, c) VALUES (?, ?, ?)", row_data)
        count += cursor.rowcount
        print(f'{count} rows added.')
        db.commit()

    except sqlite3.Error as e:
        print(f'could not insert rows: {e}')
        exit(1)

    try:
        #count rows using SELECT COUNT(*)
        cursor.execute("SELECT COUNT(*) FROM hello")
        count = cursor.fetchone()[0]
        print(f'there are {count} rows in the table.')
        
        #Get column names from SQLite meta-data table_info
        cursor.execute("PRAGMA table_info(hello);")
        row = cursor.fetchall()
        colnames = [r[1] for r in row]
        print(f'column names are: {colnames}')

        #Fetch rows using iterator
        print('\nusing iterator.')
        cursor.execute("SELECT * FROM hello LIMIT 5")
        for row in cursor:
            print(row)

        #Fetch rows using row_factory
        print('\nusing row_factory.')
        cursor.execute("SELECT * FROM hello LIMIT 5")
        cursor.row_factory = sqlite3.Row
        for row in cursor:
            print(f'as tuple: {tuple(row)}, as dict: id:{row['id']} a:{row['a']}, b:{row['b']}, c:{['c']}')

        cursor.row_factory = None #Reset row_factory

        #Fetch rows in groups of 5 using fetchmany
        print('\ngroups of 5 using fetchmany.')
        cursor.execute("SELECT * FROM hello")
        rows = cursor.fetchmany(5)
        while rows:
            for r in rows:
                print(r)
            print("====== ====== ======")
            rows = cursor.fetchmany(5)

        #Drop table and close the database
        print('\ndrop table and close connection.')
        cursor.execute("DROP TABLE IF EXISTS hello") #Cleanup if db is not :memory:
        cursor.close()
        db.close()

    except sqlite3.Error as e:
        print(f'sqlite3 error {e}')
        exit(1)

if __name__ == "__main__":
    main()


SQLite example
Connected.
Table created.
Inserting rows.
27 rows added.
there are 27 rows in the table.
column names are: ['id', 'a', 'b', 'c']

using iterator.
(1, 'one', 'two', 'three')
(2, 'two', 'three', 'four')
(3, 'three', 'four', 'five')
(4, 'four', 'five', 'six')
(5, 'five', 'six', 'seven')

using row_factory.
as tuple: (1, 'one', 'two', 'three'), as dict: id:1 a:one, b:two, c:['c']
as tuple: (2, 'two', 'three', 'four'), as dict: id:2 a:two, b:three, c:['c']
as tuple: (3, 'three', 'four', 'five'), as dict: id:3 a:three, b:four, c:['c']
as tuple: (4, 'four', 'five', 'six'), as dict: id:4 a:four, b:five, c:['c']
as tuple: (5, 'five', 'six', 'seven'), as dict: id:5 a:five, b:six, c:['c']

groups of 5 using fetchmany.
(1, 'one', 'two', 'three')
(2, 'two', 'three', 'four')
(3, 'three', 'four', 'five')
(4, 'four', 'five', 'six')
(5, 'five', 'six', 'seven')
====== ====== ======
(6, 'six', 'seven', 'eight')
(7, 'seven', 'eight', 'nine')
(8, 'eight', 'nine', 'ten')
(9, 'nine', 'ten', 'e